In [2]:
from Crypto.Util.number import *

def nextPrime(p):
    while(not isPrime(p)):
        p += 1
    return p

p = 33002887544016314031879710107963270568979015443349496461580994217348108581943462671401891805445187804473789436219797804155565167447548582923397954849996799999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999
leak1 = 23970024189101676739284999949112801596159475131875488713138494179350672457539898031864018813881524127059547346056890355337626004377168849677299388452735608521932571216168945616402953073274716141377186272923187362572945186088197355146879606612986821843373706666768942322893903428906105943365666240678544198804428017369088
leak2 = 18968478281644090200080507268714677611062523716159449265101819113233914046740738809721658379928130465004476176599403035347489355703796382914550216763662625998487554273370838080897607173662469212264589929308724920155969968302754999378875497515768504097606413596494718061943021600129084717056109413521498214039208905932800
leak3 = 24846971521346659533622446473264138521227563244935912977395904811225387611724566290176121582061549516865041576960467180732303630070352480104283169087055565855330769507536550703237906927824653705302857676851252497170817996780480787545788757588029762501569142537824866794895641164190331637129027655971847921430387098648576
leak4 = 14506384300219774277241069081690189345236549123813938708663001975576355357273770188385102465249828661430885152936340057890533616107247269132829273586451886946676551549255821786705150859518035298037172807466360938386435146531493168660580846121143769776024893444008660445579419515274295620015909587687581741666493110681600
n = 17493161302233539865959778115545849614338747587320539830958673046207703917485289344059005689614899617727057491562059564216328452528431891394964839425545058028178376719697816107862705922877310886122597831458084446977069139926783492027694957365054350790083539389958649142920896888580550434273011064581701139955036959685836337063232723340151170432920208045324320904257623616896017119310224381452189172970937417436861279136518242101847150438391226935084568440274930855362251299289608983772866874344756978071504480416710819717504154104303015133892355188631899268990010375768150730500938937147573182759644411399269734039357767059492842474140949
cipher = 16921253539344108497068632546240027709825757991466136956611001346572164679990250077204370295206837487391144086361806843608963030262072269950948128913794922674041531052955655884066609971456009776555957969815343040743446935276861391783043533446286228409288466221337064700577406994578750474068930699978389468927444219610938550365285817940713903147886965909078470774683916754388441563810469530491686011112673883268153010239483497777390409547764296300047653921679373300693423282406815477193889860589206837874689550943581455570596354670272027903696833667599311930587303088445675268598000581375688180056683387083737664908840976358250628881034642

##################################################### part1
F.<i> = GF(p^2, modulus = x^2 + 1)
PR.<a,b,c,d> = PolynomialRing(Zmod(p))
a,b,c,d = PR.gens()
X = (leak1 + a) + (leak2 + b) * i
Y = (leak3 + c) + (leak4 + d) * i
f = (
    X^3
    + Y^3
    - X^2 * Y^2
    + 1488 * X^2 * Y
    + 1488 * X * Y^2
    - 162000 * X^2
    - 162000 * Y^2
    + 40773375 * X * Y
    + 8748000000 * X
    + 8748000000 * Y
    - 157464000000000
)

f_re = 0
f_im = 0
for jj in f:
    temp = str(jj[0]).split("*i")
    if(len(temp) == 1):
        if(temp[0] == "i"):
            f_im += jj[1]
        else:
            f_re += int(temp[0])*jj[1]
    else:
        f_im += int(temp[0])*jj[1]
        if(temp[1] != ""):
            f_re += int(temp[1][2:])*jj[1]

polys = []


deg_h = (f_re+f_im).degree()
polys.append(f_re)
polys.append(f_im)
T = 2^48
M, v = Sequence(polys).coefficient_matrix()
M = M.T.dense_matrix()
R, C = M.dimensions()
B = block_matrix(
    ZZ, [[matrix.identity(C)*p, matrix.zero(C, R)], [M, matrix.identity(R)]]
)
vT = (v.T).list()

B[:,:C] *= p
for jj in range(len(vT)):
    B[:,C+jj:C+jj+1] *= T^(deg_h-vT[jj].degree())
print(B.dimensions())

res = B.LLL()
for tt in res:
    if(abs(tt[-1]) == T^deg_h):
        a,b,c,d = abs(tt[-5])//T^(deg_h-1) ,  abs(tt[-4])//T^(deg_h-1) ,  abs(tt[-3])//T^(deg_h-1) , abs(tt[-2])//T^(deg_h-1)
        if(a > 0):
            break


j2 = (leak3 + c) + (leak4 + d) * i
##################################################### part2
#Φ5
def find_neighbors_phi5(X,j_prev=None):
    R.<Y> = PolynomialRing(X.parent())
    Φ5 = (
        X^6
        + Y^6
        - X^5*Y^5 
        + 3720*X^5*Y^4 
        + 3720*X^4*Y^5 
        - 4550940*X^5*Y^3 
        - 4550940*X^3*Y^5
        + 2028551200*X^5*Y^2 
        + 2028551200*X^2*Y^5
        - 246683410950*X^5*Y
        - 246683410950*X*Y^5 
        + 1963211489280*X^5 
        + 1963211489280*Y^5 
        + 1665999364600*X^4*Y^4 
        + 107878928185336800*X^4*Y^3 
        + 107878928185336800*X^3*Y^4 
        + 383083609779811215375*X^4*Y^2 
        + 383083609779811215375*X^2*Y^4
        + 128541798906828816384000*X^4*Y 
        + 128541798906828816384000*X*Y^4  
        + 1284733132841424456253440*X^4
        + 1284733132841424456253440*Y^4
        - 441206965512914835246100*X^3*Y^3  
        + 26898488858380731577417728000*X^3*Y^2 
        + 26898488858380731577417728000*X^2*Y^3 
        - 192457934618928299655108231168000*X^3*Y
        - 192457934618928299655108231168000*X*Y^3 
        + 280244777828439527804321565297868800*X^3
        + 280244777828439527804321565297868800*Y^3
        + 5110941777552418083110765199360000*X^2*Y^2 
        + 36554736583949629295706472332656640000*X^2*Y 
        + 36554736583949629295706472332656640000*X*Y^2        
        + 6692500042627997708487149415015068467200*X^2 
        - 264073457076620596259715790247978782949376*X*Y 
        + 6692500042627997708487149415015068467200*Y^2 
        + 53274330803424425450420160273356509151232000*X 
        + 53274330803424425450420160273356509151232000*Y 
        + 141359947154721358697753474691071362751004672000
    )

    res = Φ5.roots(multiplicities=False)
    if(j_prev == None):
        return res
    else:
        return list(set(res) - set([j_prev]))
    

for j3 in find_neighbors_phi5(j2):
    p = GCD(nextPrime(int(j3[0])),n)
    if(p != 1):
        q = n // p
        d = inverse(65537,(p-1)*(q-1))
        print(long_to_bytes(int(pow(cipher,d,n))))
        break


#H&NCTF{5till_n0_n33d_to_u5e_co0p3R!XD}

/tmp/ipykernel_638/1917938532.py:57: DeprecationWarning: the function coefficient_matrix is deprecated; use coefficients_monomials instead
See https://github.com/sagemath/sage/issues/37035 for details.
  M, v = Sequence(polys).coefficient_matrix()


(46, 46)
b'H&NCTF{5till_n0_n33d_to_u5e_co0p3R!XD}'
